In [44]:
import pandas as pd
import json
file_path = "/raid/data/zyj/AgentGym/merged_agent_traj.jsonl"
# file_path = "/raid/data/datasets/AgentTraj-L/textcraft_train.json"
# file_path = "/raid/data/zyj/AgentGym/data/merged_agent_traj.jsonl"
if file_path.endswith("jsonl"):
    df = pd.read_json(file_path, lines=True)
elif file_path.endswith("json"):
    dj = json.load(open(file_path))
    df = pd.DataFrame(dj)
    del dj
df.head()


,messages
0,"[{'role': 'user', 'content': 'Interact with a ..."
1,"[{'role': 'user', 'content': 'Interact with a ..."
2,"[{'role': 'user', 'content': 'Interact with a ..."
3,"[{'role': 'user', 'content': 'Interact with a ..."
4,"[{'role': 'user', 'content': 'Interact with a ..."


In [45]:
def convert_conversation(item):
    conversations = []
    i = 0
    # 优化：直接遍历 messages 列表，更 Pythonic
    for msg in item['messages']:
        i += 1
        if msg['role'] == 'user':
            conversations.append(
                {
                    'from': 'human',
                    'value': msg['content'],
                    'loss': None  # 不计算 Loss
                }
            )
        elif msg['role'] == 'assistant':
            conversations.append(
                {
                    'from': 'gpt',
                    'value': msg['content'],
                    'loss': False if i == 2 else True
                }
            )
    return conversations

In [46]:
new_data = []
i = 0
for item in df.to_dict(orient='records'):
    i += 1
    new_data.append({
        'conversations': convert_conversation(item),
        'item_id': i
    })
new_data[0]


{'conversations': [{'from': 'human',
   'value': 'Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:"Thought: your thoughts.\nAction: your next action\n".',
   'loss': None},
  {'from': 'gpt',
   'value': "OK. I'll follow your instructions and try my best to solve the task.",
   'loss': False},
  {'from': 'human',
   'value': 'You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 17, a drawer 16, a drawer 15, a d

In [47]:
with open("/raid/data/zyj/AgentGym/data/merged_agent_traj.json", "w") as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)